 #### <p style="background-color: #EDE7F6;color:#6600ff;display: inline-block;padding:.6rem;border-radius:.5rem">Import libraries</p>

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve
from catboost import CatBoostRegressor
import warnings

 #### <p style="background-color: #EDE7F6;color:#6600ff;display: inline-block;padding:.6rem;border-radius:.5rem">Settings</p>

In [5]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100
pd.options.display.max_colwidth = 1000
# pd.options.display.precision = 5
warnings.filterwarnings('ignore')

In [6]:
def oh_encoder(column, df):
    data = df[[column]]
    oh = OneHotEncoder()
    data_oh = oh.fit_transform(data)
    print(oh.categories_[0])
    
    data_oh = data_oh.toarray()
    list_column = []
    
    for i in oh.categories_[0]:
        string = column + '_' + i
        list_column.append(string)
    data_oh = pd.DataFrame(data_oh, index=df.index, columns=list_column)
    
    df_new = pd.concat([df,data_oh], axis=1)
    df_new = df_new.drop(column, axis=1)
    return df_new
    

In [7]:
def remove_label(df, label):
    x = df.drop(label, axis=1)
    y = df[label].copy()
    
    return (x,y)

In [8]:
def dif_nan(df):
    categories = df.unique()
    for i in categories:
        if type(i) == str:
            return i

 #### <p style="background-color: #EDE7F6;color:#6600ff;display: inline-block;padding:.6rem;border-radius:.5rem">Import data files</p>

In [9]:
train_data = pd.read_csv("train_transaction.csv", index_col=0)

### <p style="background-color: #EDE7F6; color: #6600ff;margin:0; display:inline-block;padding:.6rem;border-radius:.25rem;">I.DATA PREPARATION</p>

In [10]:
y = train_data['isFraud']

In [11]:
train_data = train_data.drop('isFraud',axis=1)

#### <p style="background-color: #EDE7F6; color: #6600ff;margin:0; display:inline-block;padding:.6rem;border-radius:.25rem;">A.EDA ( Exploratory Data Analysis)</p>

In [10]:
train_data.tail(4)

,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3577536,15811049,39.50,W,10444,225.0,150.0,mastercard,224.0,debit,204.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,T,F,F,M0,F,T,F,F,F,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3577537,15811079,30.95,W,12037,595.0,150.0,mastercard,224.0,debit,231.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1

In [11]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 590540 entries, 2987000 to 3577539
Columns: 392 entries, TransactionDT to V339
dtypes: float64(376), int64(2), object(14)
memory usage: 1.7+ GB


In [ ]:
train_data.shape

In [ ]:
train_data.describe()

#### <p style="background-color: #EDE7F6; color: #6600ff;margin:0; display:inline-block;padding:.6rem;border-radius:.25rem;">B.Data Pre-processing</p>

 #### <p style="background-color: #EDE7F6;color:#6600ff;display: inline-block;padding:.6rem;border-radius:.5rem">B1.Missing Values</p>

In [12]:
for i in train_data.columns:
    if train_data[i].isna().any() == True:
        print(f'Column {i} have null values {train_data[i].isna().sum()}')

Column card2 have null values 8933
Column card3 have null values 1565
Column card4 have null values 1577
Column card5 have null values 4259
Column card6 have null values 1571
Column addr1 have null values 65706
Column addr2 have null values 65706
Column dist1 have null values 352271
Column dist2 have null values 552913
Column P_emaildomain have null values 94456
Column R_emaildomain have null values 453249
Column D1 have null values 1269
Column D2 have null values 280797
Column D3 have null values 262878
Column D4 have null values 168922
Column D5 have null values 309841
Column D6 have null values 517353
Column D7 have null values 551623
Column D8 have null values 515614
Column D9 have null values 515614
Column D10 have null values 76022
Column D11 have null values 279287
Column D12 have null values 525823
Column D13 have null values 528588
Column D14 have null values 528353
Column D15 have null values 89113
Column M1 have null values 271100
Column M2 have null values 271100
Column M3 

 #### <p style="background-color: #EDE7F6;color:#6600ff;display: inline-block;padding:.6rem;border-radius:.5rem">B2.Filling Nulls</p>

In [13]:
for i in train_data.columns:
    if train_data[i].isna().any() == True:
        if train_data[i].dtype != 'O':
            print(f'Column {i} have null values {train_data[i].isna().sum()}. Filling null values with mean {train_data[i].mean()}')
            train_data[i] = train_data[i].fillna(train_data[i].mean())
        else:
            print(f'Column {i} have null values {train_data[i].isna().sum()}. Fillin null values with U')
            train_data[i] = train_data[i].fillna('U')

Column card2 have null values 8933. Filling null values with mean 362.5554884999665
Column card3 have null values 1565. Filling null values with mean 153.19492508170975
Column card4 have null values 1577. Fillin null values with U
Column card5 have null values 4259. Filling null values with mean 199.27889697943476
Column card6 have null values 1571. Fillin null values with U
Column addr1 have null values 65706. Filling null values with mean 290.73379392341195
Column addr2 have null values 65706. Filling null values with mean 86.80063029453122
Column dist1 have null values 352271. Filling null values with mean 118.50218030881064
Column dist2 have null values 552913. Filling null values with mean 231.85542296754988
Column P_emaildomain have null values 94456. Fillin null values with U
Column R_emaildomain have null values 453249. Fillin null values with U
Column D1 have null values 1269. Filling null values with mean 94.34756843625429
Column D2 have null values 280797. Filling null value

Column V67 have null values 77096. Filling null values with mean 0.998120535053482
Column V68 have null values 77096. Filling null values with mean 0.0005336511868869828
Column V69 have null values 77096. Filling null values with mean 0.3902002944819688
Column V70 have null values 77096. Filling null values with mean 0.4079237463092372
Column V71 have null values 77096. Filling null values with mean 0.1406385116974782
Column V72 have null values 77096. Filling null values with mean 0.14512390835222536
Column V73 have null values 77096. Filling null values with mean 0.1399821596902486
Column V74 have null values 77096. Filling null values with mean 0.1521470695927891
Column V75 have null values 89164. Filling null values with mean 0.5442781465407199
Column V76 have null values 89164. Filling null values with mean 0.5875570430176156
Column V77 have null values 89164. Filling null values with mean 1.0868928708195047
Column V78 have null values 89164. Filling null values with mean 1.144462

Column V178 have null values 450909. Filling null values with mean 6.64928275239739
Column V179 have null values 450909. Filling null values with mean 4.86901189563922
Column V180 have null values 450721. Filling null values with mean 0.9241233308777776
Column V181 have null values 450909. Filling null values with mean 0.2533319964764272
Column V182 have null values 450909. Filling null values with mean 0.8562425249407366
Column V183 have null values 450909. Filling null values with mean 0.48469179480201385
Column V184 have null values 450721. Filling null values with mean 0.13236398486614837
Column V185 have null values 450721. Filling null values with mean 0.1739034036861943
Column V186 have null values 450909. Filling null values with mean 1.148799335391138
Column V187 have null values 450909. Filling null values with mean 1.8437524618458652
Column V188 have null values 450721. Filling null values with mean 1.0147547901215141
Column V189 have null values 450721. Filling null values 

Column V284 have null values 12. Filling null values with mean 0.08854279559986995
Column V285 have null values 12. Filling null values with mean 1.1676601278855532
Column V286 have null values 12. Filling null values with mean 0.03149215617210361
Column V287 have null values 12. Filling null values with mean 0.3585791020916874
Column V288 have null values 1269. Filling null values with mean 0.18434981528023608
Column V289 have null values 1269. Filling null values with mean 0.23597461948746842
Column V290 have null values 12. Filling null values with mean 1.1030112035331094
Column V291 have null values 12. Filling null values with mean 1.6598112197897474
Column V292 have null values 12. Filling null values with mean 1.2399158041616993
Column V293 have null values 12. Filling null values with mean 0.9425988268126152
Column V294 have null values 12. Filling null values with mean 2.3138631868429607
Column V295 have null values 12. Filling null values with mean 1.4334239866695568
Column V

 #### <p style="background-color: #EDE7F6;color:#6600ff;display: inline-block;padding:.6rem;border-radius:.5rem">B3.Remove Duplicates and Unnecessary Columns</p>

In [17]:
train_data.duplicated().sum()

3

#### <p style="background-color: #EDE7F6; color: #6600ff;margin:0; display:inline-block;padding:.6rem;border-radius:.25rem;">C.Data Processing</p>

#### <p style="background-color: #EDE7F6; color: #6600ff;margin:0; display:inline-block;padding:.6rem;border-radius:.25rem;">C1.Encoding Categorical</p>

In [14]:
for i in train_data.columns:
    if train_data[i].dtype == 'O':
        train_data = oh_encoder(i, train_data)
train_data

['C' 'H' 'R' 'S' 'W']
['U' 'american express' 'discover' 'mastercard' 'visa']
['U' 'charge card' 'credit' 'debit' 'debit or credit']
['U' 'aim.com' 'anonymous.com' 'aol.com' 'att.net' 'bellsouth.net'
 'cableone.net' 'centurylink.net' 'cfl.rr.com' 'charter.net' 'comcast.net'
 'cox.net' 'earthlink.net' 'embarqmail.com' 'frontier.com'
 'frontiernet.net' 'gmail' 'gmail.com' 'gmx.de' 'hotmail.co.uk'
 'hotmail.com' 'hotmail.de' 'hotmail.es' 'hotmail.fr' 'icloud.com'
 'juno.com' 'live.com' 'live.com.mx' 'live.fr' 'mac.com' 'mail.com'
 'me.com' 'msn.com' 'netzero.com' 'netzero.net' 'optonline.net'
 'outlook.com' 'outlook.es' 'prodigy.net.mx' 'protonmail.com' 'ptd.net'
 'q.com' 'roadrunner.com' 'rocketmail.com' 'sbcglobal.net' 'sc.rr.com'
 'servicios-ta.com' 'suddenlink.net' 'twc.com' 'verizon.net' 'web.de'
 'windstream.net' 'yahoo.co.jp' 'yahoo.co.uk' 'yahoo.com' 'yahoo.com.mx'
 'yahoo.de' 'yahoo.es' 'yahoo.fr' 'ymail.com']
['U' 'aim.com' 'anonymous.com' 'aol.com' 'att.net' 'bellsouth.net'
 'c

,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,dist1,dist2,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,ProductCD_C,ProductCD_H,ProductCD_R,ProductCD_S,ProductCD_W,card4_U,card4_american express,card4_discover,card4_mastercard,card4_visa,card6_U,card6_charge card,card6_credit,card6_debit,card6_debit or credit,P_emaildomain_U,P_emaildomain_aim.com,P_emaildomain_anonymous.com,P_emaildomain_aol.com,P_emaildomain_att.net,P_emaildomain_bellsouth.net,P_emaildomain_cableone.net,P_emaildomain_centurylink.net,P_emaildomain_cfl.rr.com,P_emaildomain_charter.net,P_emaildomain_comcast.net,P_emaildomain_cox.net,P_emaildomain_earthlink.net,P_emaildomain_embarqmail.com,P_emaildomain_frontier.com,P_emaildomain_frontiernet.net,P_emaildomain_gmail,P_emaildomain_gmail.com,P_emaildomain_gmx.de,P_emaildomain_hotmail.co.uk,P_emaildomain_hotmail.com,P_emaildomain_hotmail.de,P_emaildomain_hotmail.es,P_emaildomain_hotmail.fr,P_emaildomain_icloud.com,P_emaildomain_juno.com,P_emaildomain_live.com,P_emaildomain_live.com.mx,P_emaildomain_live.fr,P_emaildomain_mac.com,P_emaildomain_mail.com,P_emaildomain_me.com,P_emaildomain_msn.com,P_emaildomain_netzero.com,P_emaildomain_netzero.net,P_emaildomain_optonline.net,P_emaildomain_outlook.com,P_emaildomain_outlook.es,P_emaildomain_prodigy.net.mx,P_emaildomain_protonmail.com,P_emaildomain_ptd.net,P_emaildomain_q.com,P_emaildomain_roadrunner.com,P_emaildomain_rocketmail.com,P_emaildomain_sbcglobal.net,P_emaildomain_sc.rr.com,P_emaildomain_servicios-ta.com,P_emaildomain_suddenlink.net,P_emaildomain_twc.com,P_emaildomain_verizon.net,P_emaildomain_web.de,P_emaildomain_windstream.net,P_emaildomain_yahoo.co.jp,P_emaildomain_yahoo.co.uk,P_emaildomain_yahoo.com,P_emaildomain_yahoo.com.mx,P_emaildomain_yahoo.de,P_emaildomain_yahoo.es,P_emaildomain_yahoo.fr,P_emaildomain_ymail.com,R_emaildomain_U,R_emaildomain_aim.com,R_emaildomain_anonymous.com,R_emaildomain_aol.com,R_emaildomain_att.net,R_emaildomain_bellsouth.net,R_emaildomain_cableone.net,R_emaildomain_centurylink.net,R_emaildomain_cfl.rr.com,R_emaildomain_charter.net,R_emaildomain_comcast.net,R_emaildomain_cox.net,R_emaildomain_earthlink.net,R_emaildomain_embarqmail.com,R_emaildomain_frontier.com,R_emaildomain_frontiernet.net,R_emaildomain_gmail,R_emaildomain_gmail.com,R_emaildomain_gmx.de,R_emaildomain_hotmail.co.uk,R_emaildomain_hotmail.com

In [15]:
# train_set, val_set = train_test_split(train_data, test_size=0.3, random_state=42, shuffle=True, stratify=None)
# val_set, test_set = train_test_split(val_set, test_size=0.5, random_state=42, shuffle=True, stratify=None)

In [14]:
# x_train, y_train = remove_label(train_set, "isFraud")
# x_val, y_val = remove_label(val_set, "isFraud")
# x_test, y_test = remove_label(test_set, "isFraud")

In [18]:
train_data["card6_debit or credit"]

KeyError: 'card6_debit or credit'

#### <p style="background-color: #EDE7F6; color: #6600ff;margin:0; display:inline-block;padding:.6rem;border-radius:.25rem;">C2.Delete unnecessary columns</p>

In [19]:
train_data = train_data.drop("card6_debit or credit", axis=1)


KeyError: "['card6_debit or credit'] not found in axis"

### <p style="background-color: #EDE7F6; color: #6600ff;margin:0; display:inline-block;padding:.6rem;border-radius:.25rem;">II.MODEL ENGINEERING</p>

In [18]:
X = train_data.copy()
# y = train_data['isFraud']

In [16]:
# train_data = train_data.drop('isFraud',axis=1)

In [19]:
X.shape

(590540, 541)

In [20]:
train_data.shape

(590540, 541)

In [21]:
y.shape

(590540,)

In [21]:
x_train,x_val, y_train,y_val=train_test_split(X,y,test_size=0.2,random_state=42)

 #### <p style="background-color: #EDE7F6;color:#6600ff;display: inline-block;padding:.6rem;border-radius:.5rem">Import libraries</p>

In [51]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

#### <p style="background-color: #EDE7F6; color: #6600ff;margin:0; display:inline-block;padding:.6rem;border-radius:.25rem;">A.Model Training</p>

#### <p style="background-color: #EDE7F6; color: #6600ff;margin:0; display:inline-block;padding:.6rem;border-radius:.25rem;">A1.XGBRegressor</p>

In [52]:
# model_XGB = XGBClassifier(objective="binary:logistic", random_state=42)
model_XGB = XGBRegressor(random_state=42)

In [53]:
model_XGB.fit(x_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [54]:
predictionsXGB = model_XGB.predict(x_val)

#### <p style="background-color: #EDE7F6; color: #6600ff;margin:0; display:inline-block;padding:.6rem;border-radius:.25rem;">XGBRegressor - Model Evaluation</p>

In [56]:
print(f'MAE: {mean_absolute_error(predictionsXGB, y_val)}')
print(f'Train model score: {model_XGB.score(x_train, y_train)}')
print(f'Valid model score: {model_XGB.score(x_val, y_val)}')

MAE: 0.04116216137089547
Train model score: 0.5248192595001524
Valid model score: 0.4736229827641717


In [22]:
y_pred = model_XGB.predict(x_test)
y_pred

NameError: name 'x_test' is not defined

In [36]:
print("ROC_AUC: {0}".format(roc_auc_score(y_test, y_pred)))
print("ROC_CURVE: {0}".format(roc_curve(y_test, y_pred)))

ROC_AUC: 0.7463711121850095
ROC_CURVE: (array([0.        , 0.00156877, 1.        ]), array([0.      , 0.494311, 1.      ]), array([inf,  1.,  0.]))


#### <p style="background-color: #EDE7F6; color: #6600ff;margin:0; display:inline-block;padding:.6rem;border-radius:.25rem;">A2.CatBoostRegressor</p>

In [45]:
cat = CatBoostRegressor(iterations=300, custom_metric=['AUC'], eval_metric='AUC', learning_rate=0.06, l2_leaf_reg=500,  min_data_in_leaf=100)
cat.fit(x_train, y=y_train, eval_set=(x_val, y_val), plot=True, use_best_model=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	test: 0.7346003	best: 0.7346003 (0)	total: 246ms	remaining: 1m 13s
1:	test: 0.7552642	best: 0.7552642 (1)	total: 334ms	remaining: 49.7s
2:	test: 0.7771668	best: 0.7771668 (2)	total: 427ms	remaining: 42.3s
3:	test: 0.7687092	best: 0.7771668 (2)	total: 519ms	remaining: 38.4s
4:	test: 0.7790029	best: 0.7790029 (4)	total: 632ms	remaining: 37.3s
5:	test: 0.7805009	best: 0.7805009 (5)	total: 727ms	remaining: 35.6s
6:	test: 0.7793051	best: 0.7805009 (5)	total: 823ms	remaining: 34.4s
7:	test: 0.7832269	best: 0.7832269 (7)	total: 932ms	remaining: 34s
8:	test: 0.7936550	best: 0.7936550 (8)	total: 1.07s	remaining: 34.5s
9:	test: 0.7956473	best: 0.7956473 (9)	total: 1.18s	remaining: 34.2s
10:	test: 0.8172130	best: 0.8172130 (10)	total: 1.27s	remaining: 33.3s
11:	test: 0.8183206	best: 0.8183206 (11)	total: 1.36s	remaining: 32.7s
12:	test: 0.8190365	best: 0.8190365 (12)	total: 1.46s	remaining: 32.3s
13:	test: 0.8185563	best: 0.8190365 (12)	total: 1.55s	remaining: 31.6s
14:	test: 0.8190436	best: 0

116:	test: 0.8763255	best: 0.8763255 (116)	total: 10.7s	remaining: 16.7s
117:	test: 0.8763333	best: 0.8763333 (117)	total: 10.8s	remaining: 16.6s
118:	test: 0.8764969	best: 0.8764969 (118)	total: 10.9s	remaining: 16.5s
119:	test: 0.8765096	best: 0.8765096 (119)	total: 10.9s	remaining: 16.4s
120:	test: 0.8767439	best: 0.8767439 (120)	total: 11s	remaining: 16.3s
121:	test: 0.8767536	best: 0.8767536 (121)	total: 11.1s	remaining: 16.2s
122:	test: 0.8771700	best: 0.8771700 (122)	total: 11.2s	remaining: 16.1s
123:	test: 0.8771953	best: 0.8771953 (123)	total: 11.3s	remaining: 16s
124:	test: 0.8772153	best: 0.8772153 (124)	total: 11.4s	remaining: 15.9s
125:	test: 0.8772909	best: 0.8772909 (125)	total: 11.5s	remaining: 15.8s
126:	test: 0.8774266	best: 0.8774266 (126)	total: 11.6s	remaining: 15.8s
127:	test: 0.8774733	best: 0.8774733 (127)	total: 11.7s	remaining: 15.7s
128:	test: 0.8774804	best: 0.8774804 (128)	total: 11.7s	remaining: 15.6s
129:	test: 0.8774850	best: 0.8774850 (129)	total: 11.8s

230:	test: 0.8811394	best: 0.8811395 (229)	total: 19.8s	remaining: 5.9s
231:	test: 0.8811396	best: 0.8811396 (231)	total: 19.8s	remaining: 5.81s
232:	test: 0.8811398	best: 0.8811398 (232)	total: 19.9s	remaining: 5.72s
233:	test: 0.8811398	best: 0.8811398 (232)	total: 20s	remaining: 5.64s
234:	test: 0.8811405	best: 0.8811405 (234)	total: 20.1s	remaining: 5.55s
235:	test: 0.8811410	best: 0.8811410 (235)	total: 20.1s	remaining: 5.46s
236:	test: 0.8811411	best: 0.8811411 (236)	total: 20.2s	remaining: 5.37s
237:	test: 0.8811410	best: 0.8811411 (236)	total: 20.3s	remaining: 5.29s
238:	test: 0.8811411	best: 0.8811411 (236)	total: 20.4s	remaining: 5.2s
239:	test: 0.8811411	best: 0.8811411 (239)	total: 20.4s	remaining: 5.11s
240:	test: 0.8811418	best: 0.8811418 (240)	total: 20.5s	remaining: 5.02s
241:	test: 0.8811419	best: 0.8811419 (241)	total: 20.6s	remaining: 4.94s
242:	test: 0.8811419	best: 0.8811419 (242)	total: 20.7s	remaining: 4.85s
243:	test: 0.8811420	best: 0.8811420 (243)	total: 20.8s

In [46]:
predictionsCAT = cat.predict(x_val)

#### <p style="background-color: #EDE7F6; color: #6600ff;margin:0; display:inline-block;padding:.6rem;border-radius:.25rem;">CatBoostRegressor - Model Evaluation</p>

In [47]:
print(f'MAE: {mean_absolute_error(predictionsCAT, y_val)}')
print(f'Train model score: {cat.score(x_train, y_train)}')
print(f'Valid model score: {cat.score(x_val, y_val)}')

MAE: 0.04967474195662464
Train model score: 0.30643342631984194
Valid model score: 0.3086006365234506


In [48]:
y_pred = cat.predict(x_test)
y_pred

NameError: name 'x_test' is not defined

In [21]:
print("ROC_AUC: {0}".format(roc_auc_score(y_test, y_pred)))
print("ROC_CURVE: {0}".format(roc_curve(y_test, y_pred)))

ROC_AUC: 0.8952304054651378
ROC_CURVE: (array([0.        , 0.        , 0.        , ..., 0.99717855, 0.99717855,
       1.        ]), array([0.00000000e+00, 3.16055626e-04, 9.03919090e-02, ...,
       9.99683944e-01, 1.00000000e+00, 1.00000000e+00]), array([        inf,  1.23731712,  0.94089917, ..., -0.02035892,
       -0.02036819, -0.09055036]))


In [ ]:
test_data = pd.read_csv("test_transaction.csv", index_col=0)

In [29]:
test_data.shape

(506691, 392)

In [30]:
test_data.head(3)

,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3663549,18403224,31.95,W,10409,111.0,150.0,visa,226.0,debit,170.0,87.0,1.0,NaN,gmail.com,NaN,6.0,6.0,0.0,0.0,3.0,4.0,0.0,0.0,6.0,0.0,5.0,1.0,115.0,6.0,419.0,419.0,27.0,398.0,27.0,NaN,NaN,NaN,NaN,418.0,203.0,NaN,NaN,NaN,409.0,T,T,F,NaN,NaN,F,T,T,T,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,47.950001,0.0,0.0,47.950001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,47.950001,0.0,0.0,47.950001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3663550,18403263,49.00,W,4272,111.0,150.0,visa,226.0,debit,299.0,87.0,4.0,NaN,aol.com,NaN,3.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0

In [31]:
for i in test_data.columns:
    if test_data[i].isna().any() == True:
        if test_data[i].dtype != 'O':
            print(f'Column {i} have null values {test_data[i].isna().sum()}. Filling null values with mean {test_data[i].mean()}')
            test_data[i] = test_data[i].fillna(test_data[i].mean())
        else:
            print(f'Column {i} have null values {test_data[i].isna().sum()}. Fillin null values with letter U')
            test_data[i] = test_data[i].fillna('U')

Column card2 have null values 8654. Filling null values with mean 363.7353790983401
Column card3 have null values 3002. Filling null values with mean 153.5434087303872
Column card4 have null values 3086. Fillin null values with letter U
Column card5 have null values 4547. Filling null values with mean 200.1629751625032
Column card6 have null values 3007. Fillin null values with letter U
Column addr1 have null values 65609. Filling null values with mean 291.8465138001551
Column addr2 have null values 65609. Filling null values with mean 86.72341197328387
Column dist1 have null values 291217. Filling null values with mean 87.065270055784
Column dist2 have null values 470255. Filling null values with mean 237.17504665715228
Column P_emaildomain have null values 69192. Fillin null values with letter U
Column R_emaildomain have null values 370821. Fillin null values with letter U
Column C1 have null values 3. Filling null values with mean 10.093211206896552
Column C2 have null values 3. Fil

Column V60 have null values 12899. Filling null values with mean 0.2804075400168492
Column V61 have null values 12899. Filling null values with mean 0.879979829563865
Column V62 have null values 12899. Filling null values with mean 0.9128620957812196
Column V63 have null values 12899. Filling null values with mean 0.2568004341909144
Column V64 have null values 12899. Filling null values with mean 0.2795266022940833
Column V65 have null values 12899. Filling null values with mean 0.9999068433672478
Column V66 have null values 12899. Filling null values with mean 1.0154396993065906
Column V67 have null values 12899. Filling null values with mean 1.031713758019571
Column V68 have null values 12899. Filling null values with mean 0.023169674680837277
Column V69 have null values 12899. Filling null values with mean 0.33938581426997605
Column V70 have null values 12899. Filling null values with mean 0.3537987654721016
Column V71 have null values 12899. Filling null values with mean 0.25873849

Column V207 have null values 369957. Filling null values with mean 46.93674297415618
Column V208 have null values 370316. Filling null values with mean 9.031152322931316
Column V209 have null values 370316. Filling null values with mean 28.00202033014564
Column V210 have null values 370316. Filling null values with mean 14.891452859406293
Column V211 have null values 369957. Filling null values with mean 138.2944867107418
Column V212 have null values 369957. Filling null values with mean 237.50705260473066
Column V213 have null values 369957. Filling null values with mean 165.11739109953717
Column V214 have null values 369957. Filling null values with mean 63.85933691133121
Column V215 have null values 369957. Filling null values with mean 137.66673816657624
Column V216 have null values 369957. Filling null values with mean 98.2277131372622
Column V217 have null values 379963. Filling null values with mean 0.9872167161164068
Column V218 have null values 379963. Filling null values with

Column V315 have null values 6031. Filling null values with mean 26.730427547521234
Column V316 have null values 3. Filling null values with mean 42.07788688632079
Column V317 have null values 3. Filling null values with mean 83.80833728463122
Column V318 have null values 3. Filling null values with mean 56.090775902586806
Column V319 have null values 3. Filling null values with mean 27.620156205899086
Column V320 have null values 3. Filling null values with mean 46.83562932631056
Column V321 have null values 3. Filling null values with mean 36.772511327291255
Column V322 have null values 430260. Filling null values with mean 0.43668145124360536
Column V323 have null values 430260. Filling null values with mean 1.2161819157148277
Column V324 have null values 430260. Filling null values with mean 0.6935144116915911
Column V325 have null values 430260. Filling null values with mean 0.04792557993484319
Column V326 have null values 430260. Filling null values with mean 0.4826575604139682
C

In [32]:
for i in test_data.columns:
    if test_data[i].dtype == 'O':
        test_data = oh_encoder(i, test_data)
test_data

['C' 'H' 'R' 'S' 'W']
['U' 'american express' 'discover' 'mastercard' 'visa']
['U' 'charge card' 'credit' 'debit']
['U' 'aim.com' 'anonymous.com' 'aol.com' 'att.net' 'bellsouth.net'
 'cableone.net' 'centurylink.net' 'cfl.rr.com' 'charter.net' 'comcast.net'
 'cox.net' 'earthlink.net' 'embarqmail.com' 'frontier.com'
 'frontiernet.net' 'gmail' 'gmail.com' 'gmx.de' 'hotmail.co.uk'
 'hotmail.com' 'hotmail.de' 'hotmail.es' 'hotmail.fr' 'icloud.com'
 'juno.com' 'live.com' 'live.com.mx' 'live.fr' 'mac.com' 'mail.com'
 'me.com' 'msn.com' 'netzero.com' 'netzero.net' 'optonline.net'
 'outlook.com' 'outlook.es' 'prodigy.net.mx' 'protonmail.com' 'ptd.net'
 'q.com' 'roadrunner.com' 'rocketmail.com' 'sbcglobal.net' 'sc.rr.com'
 'scranton.edu' 'servicios-ta.com' 'suddenlink.net' 'twc.com'
 'verizon.net' 'web.de' 'windstream.net' 'yahoo.co.jp' 'yahoo.co.uk'
 'yahoo.com' 'yahoo.com.mx' 'yahoo.de' 'yahoo.es' 'yahoo.fr' 'ymail.com']
['U' 'aim.com' 'anonymous.com' 'aol.com' 'att.net' 'bellsouth.net'
 'cabl

,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,dist1,dist2,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,ProductCD_C,ProductCD_H,ProductCD_R,ProductCD_S,ProductCD_W,card4_U,card4_american express,card4_discover,card4_mastercard,card4_visa,card6_U,card6_charge card,card6_credit,card6_debit,P_emaildomain_U,P_emaildomain_aim.com,P_emaildomain_anonymous.com,P_emaildomain_aol.com,P_emaildomain_att.net,P_emaildomain_bellsouth.net,P_emaildomain_cableone.net,P_emaildomain_centurylink.net,P_emaildomain_cfl.rr.com,P_emaildomain_charter.net,P_emaildomain_comcast.net,P_emaildomain_cox.net,P_emaildomain_earthlink.net,P_emaildomain_embarqmail.com,P_emaildomain_frontier.com,P_emaildomain_frontiernet.net,P_emaildomain_gmail,P_emaildomain_gmail.com,P_emaildomain_gmx.de,P_emaildomain_hotmail.co.uk,P_emaildomain_hotmail.com,P_emaildomain_hotmail.de,P_emaildomain_hotmail.es,P_emaildomain_hotmail.fr,P_emaildomain_icloud.com,P_emaildomain_juno.com,P_emaildomain_live.com,P_emaildomain_live.com.mx,P_emaildomain_live.fr,P_emaildomain_mac.com,P_emaildomain_mail.com,P_emaildomain_me.com,P_emaildomain_msn.com,P_emaildomain_netzero.com,P_emaildomain_netzero.net,P_emaildomain_optonline.net,P_emaildomain_outlook.com,P_emaildomain_outlook.es,P_emaildomain_prodigy.net.mx,P_emaildomain_protonmail.com,P_emaildomain_ptd.net,P_emaildomain_q.com,P_emaildomain_roadrunner.com,P_emaildomain_rocketmail.com,P_emaildomain_sbcglobal.net,P_emaildomain_sc.rr.com,P_emaildomain_scranton.edu,P_emaildomain_servicios-ta.com,P_emaildomain_suddenlink.net,P_emaildomain_twc.com,P_emaildomain_verizon.net,P_emaildomain_web.de,P_emaildomain_windstream.net,P_emaildomain_yahoo.co.jp,P_emaildomain_yahoo.co.uk,P_emaildomain_yahoo.com,P_emaildomain_yahoo.com.mx,P_emaildomain_yahoo.de,P_emaildomain_yahoo.es,P_emaildomain_yahoo.fr,P_emaildomain_ymail.com,R_emaildomain_U,R_emaildomain_aim.com,R_emaildomain_anonymous.com,R_emaildomain_aol.com,R_emaildomain_att.net,R_emaildomain_bellsouth.net,R_emaildomain_cableone.net,R_emaildomain_centurylink.net,R_emaildomain_cfl.rr.com,R_emaildomain_charter.net,R_emaildomain_comcast.net,R_emaildomain_cox.net,R_emaildomain_earthlink.net,R_emaildomain_embarqmail.com,R_emaildomain_frontier.com,R_emaildomain_frontiernet.net,R_emaildomain_gmail,R_emaildomain_gmail.com,R_emaildomain_gmx.de,R_emaildomain_hotmail.co.uk,R_emaildomain_hotmai

In [33]:
train_data.head(3)

,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,dist1,dist2,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,ProductCD_C,ProductCD_H,ProductCD_R,ProductCD_S,ProductCD_W,card4_U,card4_american express,card4_discover,card4_mastercard,card4_visa,card6_U,card6_charge card,card6_credit,card6_debit,P_emaildomain_U,P_emaildomain_aim.com,P_emaildomain_anonymous.com,P_emaildomain_aol.com,P_emaildomain_att.net,P_emaildomain_bellsouth.net,P_emaildomain_cableone.net,P_emaildomain_centurylink.net,P_emaildomain_cfl.rr.com,P_emaildomain_charter.net,P_emaildomain_comcast.net,P_emaildomain_cox.net,P_emaildomain_earthlink.net,P_emaildomain_embarqmail.com,P_emaildomain_frontier.com,P_emaildomain_frontiernet.net,P_emaildomain_gmail,P_emaildomain_gmail.com,P_emaildomain_gmx.de,P_emaildomain_hotmail.co.uk,P_emaildomain_hotmail.com,P_emaildomain_hotmail.de,P_emaildomain_hotmail.es,P_emaildomain_hotmail.fr,P_emaildomain_icloud.com,P_emaildomain_juno.com,P_emaildomain_live.com,P_emaildomain_live.com.mx,P_emaildomain_live.fr,P_emaildomain_mac.com,P_emaildomain_mail.com,P_emaildomain_me.com,P_emaildomain_msn.com,P_emaildomain_netzero.com,P_emaildomain_netzero.net,P_emaildomain_optonline.net,P_emaildomain_outlook.com,P_emaildomain_outlook.es,P_emaildomain_prodigy.net.mx,P_emaildomain_protonmail.com,P_emaildomain_ptd.net,P_emaildomain_q.com,P_emaildomain_roadrunner.com,P_emaildomain_rocketmail.com,P_emaildomain_sbcglobal.net,P_emaildomain_sc.rr.com,P_emaildomain_servicios-ta.com,P_emaildomain_suddenlink.net,P_emaildomain_twc.com,P_emaildomain_verizon.net,P_emaildomain_web.de,P_emaildomain_windstream.net,P_emaildomain_yahoo.co.jp,P_emaildomain_yahoo.co.uk,P_emaildomain_yahoo.com,P_emaildomain_yahoo.com.mx,P_emaildomain_yahoo.de,P_emaildomain_yahoo.es,P_emaildomain_yahoo.fr,P_emaildomain_ymail.com,R_emaildomain_U,R_emaildomain_aim.com,R_emaildomain_anonymous.com,R_emaildomain_aol.com,R_emaildomain_att.net,R_emaildomain_bellsouth.net,R_emaildomain_cableone.net,R_emaildomain_centurylink.net,R_emaildomain_cfl.rr.com,R_emaildomain_charter.net,R_emaildomain_comcast.net,R_emaildomain_cox.net,R_emaildomain_earthlink.net,R_emaildomain_embarqmail.com,R_emaildomain_frontier.com,R_emaildomain_frontiernet.net,R_emaildomain_gmail,R_emaildomain_gmail.com,R_emaildomain_gmx.de,R_emaildomain_hotmail.co.uk,R_emaildomain_hotmail.com,R_emaildomain_hotmail

In [34]:
test_data.head(3)

,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,dist1,dist2,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,ProductCD_C,ProductCD_H,ProductCD_R,ProductCD_S,ProductCD_W,card4_U,card4_american express,card4_discover,card4_mastercard,card4_visa,card6_U,card6_charge card,card6_credit,card6_debit,P_emaildomain_U,P_emaildomain_aim.com,P_emaildomain_anonymous.com,P_emaildomain_aol.com,P_emaildomain_att.net,P_emaildomain_bellsouth.net,P_emaildomain_cableone.net,P_emaildomain_centurylink.net,P_emaildomain_cfl.rr.com,P_emaildomain_charter.net,P_emaildomain_comcast.net,P_emaildomain_cox.net,P_emaildomain_earthlink.net,P_emaildomain_embarqmail.com,P_emaildomain_frontier.com,P_emaildomain_frontiernet.net,P_emaildomain_gmail,P_emaildomain_gmail.com,P_emaildomain_gmx.de,P_emaildomain_hotmail.co.uk,P_emaildomain_hotmail.com,P_emaildomain_hotmail.de,P_emaildomain_hotmail.es,P_emaildomain_hotmail.fr,P_emaildomain_icloud.com,P_emaildomain_juno.com,P_emaildomain_live.com,P_emaildomain_live.com.mx,P_emaildomain_live.fr,P_emaildomain_mac.com,P_emaildomain_mail.com,P_emaildomain_me.com,P_emaildomain_msn.com,P_emaildomain_netzero.com,P_emaildomain_netzero.net,P_emaildomain_optonline.net,P_emaildomain_outlook.com,P_emaildomain_outlook.es,P_emaildomain_prodigy.net.mx,P_emaildomain_protonmail.com,P_emaildomain_ptd.net,P_emaildomain_q.com,P_emaildomain_roadrunner.com,P_emaildomain_rocketmail.com,P_emaildomain_sbcglobal.net,P_emaildomain_sc.rr.com,P_emaildomain_scranton.edu,P_emaildomain_servicios-ta.com,P_emaildomain_suddenlink.net,P_emaildomain_twc.com,P_emaildomain_verizon.net,P_emaildomain_web.de,P_emaildomain_windstream.net,P_emaildomain_yahoo.co.jp,P_emaildomain_yahoo.co.uk,P_emaildomain_yahoo.com,P_emaildomain_yahoo.com.mx,P_emaildomain_yahoo.de,P_emaildomain_yahoo.es,P_emaildomain_yahoo.fr,P_emaildomain_ymail.com,R_emaildomain_U,R_emaildomain_aim.com,R_emaildomain_anonymous.com,R_emaildomain_aol.com,R_emaildomain_att.net,R_emaildomain_bellsouth.net,R_emaildomain_cableone.net,R_emaildomain_centurylink.net,R_emaildomain_cfl.rr.com,R_emaildomain_charter.net,R_emaildomain_comcast.net,R_emaildomain_cox.net,R_emaildomain_earthlink.net,R_emaildomain_embarqmail.com,R_emaildomain_frontier.com,R_emaildomain_frontiernet.net,R_emaildomain_gmail,R_emaildomain_gmail.com,R_emaildomain_gmx.de,R_emaildomain_hotmail.co.uk,R_emaildomain_hotmai

In [35]:
# train_data = train_data.drop("card6_debit or credit", axis=1)
test_data = test_data.drop("P_emaildomain_scranton.edu", axis=1)

In [36]:
test_data["P_emaildomain_scranton.edu"]

KeyError: 'P_emaildomain_scranton.edu'

In [44]:
train_data["card6_debit or credit"]

TransactionID
2987000    0.0
2987001    0.0
2987002    0.0
2987003    0.0
2987004    0.0
          ... 
3577535    0.0
3577536    0.0
3577537    0.0
3577538    0.0
3577539    0.0
Name: card6_debit or credit, Length: 590540, dtype: float64

In [34]:
# train_data = train_data.drop('isFraud',axis=1)

In [57]:
submission_2 = model_XGB.predict(test_data)
submission_2

array([-0.02110321,  0.01984049,  0.06426917, ...,  0.00026324,
        0.00509127,  0.01352089], dtype=float32)

In [49]:
submission = cat.predict(test_data)
submission

array([0.02103903, 0.01114332, 0.01047761, ..., 0.00895405, 0.01227827,
       0.02113071])

In [58]:
submission_2 = pd.DataFrame(submission_2, index=test_data.index, columns=['isFraud'])
submission_2

,isFraud
TransactionID,
3663549,-0.021103
3663550,0.019840
3663551,0.064269
3663552,-0.024754
3663553,-0.023100
...,...
4170235,0.004784
4170236,0.188595
4170237,0.000263


In [50]:
submission = pd.DataFrame(submission, index=test_data.index, columns=['isFraud'])
submission

,isFraud
TransactionID,
3663549,0.021039
3663550,0.011143
3663551,0.010478
3663552,0.029218
3663553,0.025732
...,...
4170235,0.024992
4170236,0.021076
4170237,0.008954


In [29]:
submission.to_csv("submission.csv")